# Customer_Segmentation_Olist and Northwind Sales

Purpose: Produce clean, auditable transaction-level datasets for customer segmentation. No modeling is performed in this notebook.

In [1]:
# Import necessary libraries
import pandas as pd

In [2]:
# Step 1: data_integration_cleaning
# Step 1.1: Olist Data Cleaning
# Primary segmentation dataset
# Step 1.1.1 — Load Authoritative Tables (Only)
# Olist tables (authoritative only)
orders = pd.read_csv("../data/raw/olist/olist_orders_dataset.csv")
customers = pd.read_csv("../data/raw/olist/olist_customers_dataset.csv")
payments = pd.read_csv("../data/raw/olist/olist_order_payments_dataset.csv")
order_items = pd.read_csv("../data/raw/olist/olist_order_items_dataset.csv")  # validation only

In [3]:
# Step 1.1.2 — Mandatory Status Filtering (Delivered Only)
orders_delivered = orders.loc[
    orders["order_status"] == "delivered"
].copy()

In [4]:
# Step 1.1.3 — Timestamp Normalization (Segmentation Anchor)
orders_delivered["order_purchase_timestamp"] = pd.to_datetime(
    orders_delivered["order_purchase_timestamp"],
    errors="coerce"
)

In [5]:
# Step 1.1.4 — Join Orders → Customers
orders_customers = orders_delivered.merge(
    customers[["customer_id", "customer_unique_id"]],
    on="customer_id",
    how="left",
    validate="many_to_one"
)

In [6]:
# Step 1.1.5 — Join Orders → Payments
orders_payments = orders_customers.merge(
    payments[["order_id", "payment_value"]],
    on="order_id",
    how="left",
    validate="one_to_many"
)

In [7]:
# Step 1.1.6 — Payment Aggregation (Order-Level)
# Multiple payment records per order are summed
# This is expected behavior in Olist (split payments)
orders_payments_agg = (
    orders_payments
    .groupby(
        ["customer_unique_id", "order_id", "order_purchase_timestamp"],
        as_index=False
    )["payment_value"]
    .sum()
)

In [8]:
# Step 1.1.7 — Order Items Validation (Structural QA Only)
# Orders missing items (should be zero or negligible)
orders_without_items = (
    orders_payments_agg
    .merge(
        order_items[["order_id"]].drop_duplicates(),
        on="order_id",
        how="left",
        indicator=True
    )
    .query("_merge == 'left_only'")
)

len(orders_without_items)

0

In [9]:
# Step 1.1.8 — Final Olist Cleaned Artifact
olist_orders_cleaned = orders_payments_agg.rename(
    columns={
        "order_purchase_timestamp": "order_date"
    }
)[
    ["customer_unique_id", "order_id", "order_date", "payment_value"]
]

In [10]:
# Step 1.1.9 — Hard Validations
# One row per order
assert olist_orders_cleaned["order_id"].is_unique

# Delivered orders only
assert olist_orders_cleaned.shape[0] == orders_payments_agg.shape[0]

# No missing monetary values
assert olist_orders_cleaned["payment_value"].notna().all()

# No missing customers
assert olist_orders_cleaned["customer_unique_id"].notna().all()
assert orders_delivered["order_purchase_timestamp"].notna().all()

In [11]:
# Step 1.2 — Northwind Data Cleaning
#Secondary benchmark dataset (independent)
# Step 1.2.1 — Load Authoritative Tables
nw_orders = pd.read_csv("../data/raw/northwind/orders.csv")
nw_order_details = pd.read_csv("../data/raw/northwind/order_details.csv")
nw_customers = pd.read_csv("../data/raw/northwind/customers.csv", encoding="latin1")

In [12]:
# Step 1.2.2 — Revenue Construction (Auditable Formula)
nw_order_details["line_revenue"] = (
    nw_order_details["unitPrice"]
    * nw_order_details["quantity"]
    * (1 - nw_order_details["discount"])
)

In [13]:
# Step 1.2.3 — Aggregate to Order Level
nw_order_revenue = (
    nw_order_details
    .groupby("orderID", as_index=False)["line_revenue"]
    .sum()
    .rename(columns={"line_revenue": "order_revenue"})
)

In [14]:
# Step 1.2.4 — Join Orders + Customers
nw_orders["orderDate"] = pd.to_datetime(
    nw_orders["orderDate"],
    errors="coerce"
)

northwind_orders_cleaned = (
    nw_orders
    .merge(nw_order_revenue, on="orderID", how="inner")
    .merge(
        nw_customers[["customerID"]],
        on="customerID",
        how="left",
        validate="many_to_one"
    )[
        ["customerID", "orderID", "orderDate", "order_revenue"]
    ]
)

In [15]:
# Step 1.2.5 — Hard Validations
assert northwind_orders_cleaned["orderID"].is_unique
assert northwind_orders_cleaned["order_revenue"].notna().all()
assert northwind_orders_cleaned["customerID"].notna().all()

In [16]:
#Saved clean 
olist_orders_cleaned.to_csv(
    "../data/processed/olist_orders_cleaned.csv",
    index=False
)

northwind_orders_cleaned.to_csv(
    "../data/processed/northwind_orders_cleaned.csv",
    index=False
)